In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 16.7 MB 247 kB/s 
     |████████████████████████████████| 6.3 MB 39.6 MB/s 


In [ ]:
#import the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
import numpy as np
from math import sqrt
import statsmodels.formula.api as smf

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#set up the metric calculations
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [ ]:
#read in the cdatasub from the first week
cdatasub = pd.read_csv("london_flows_index.csv", index_col=0)
for index, row in cdatasub.iterrows():
    if row["distance"]==0 or row["population"]==0 or row["jobs"]==0 or row["flows"]==0:
        cdatasub.drop(index, inplace=True)

In [ ]:
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance
index,,,,,,
2,Abbey Road,Beckton,1,599,442,8510.121774
3,Abbey Road,Blackwall,3,599,665,3775.448872
4,Abbey Road,Canary Wharf,1,599,58772,5086.514220
5,Abbey Road,Canning Town,37,599,15428,2228.923167
6,Abbey Road,Crossharbour,1,599,1208,6686.475560
8,Abbey Road,Cutty Sark,2,599,1748,8503.898909
9,Abbey Road,Cyprus,7,599,850,6532.099618
10,Abbey Road,Devons Road,1,599,611,3958.324171
11,Abbey Road,East India,2,599,1522,3384.141666


In [ ]:
for index, row in cdatasub.iterrows():
   # print(type(row["flows"]))
    
    if row["flows"]==0 or row["distance"]==0:
        #print("True",index)
        cdatasub.drop(index, inplace=True)

In [ ]:
#create the formula (the "-1" indicates no intercept in the regression model).
#dbl_form = 'Total ~ Dest + Orig + log_Dist-1'
####classical model######
x_variables = ["distance",]
for x in x_variables:
    cdatasub[f"log_{x}"] = np.log(cdatasub[x])
dbl_form = 'flows ~ station_destination + station_origin + log_distance-1'
#run a doubly constrained sim
doubSim = smf.glm(formula = dbl_form, data=cdatasub, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(doubSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                43945
Model:                            GLM   Df Residuals:                    43149
Model Family:                 Poisson   Df Model:                          795
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -8.6312e+05
Date:                Mon, 09 May 2022   Deviance:                   1.5540e+06
Time:                        09:56:42   Pearson chi2:                 2.10e+06
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------

And the various flows and goodness-of-fit statistics?

In [ ]:
#get the estimates
cdatasub["doubsimfitted"] = np.round(doubSim.mu)


In [ ]:
CalcRSqaured(cdatasub["flows"],cdatasub["doubsimfitted"])

0.41893772760573356

In [ ]:
CalcRMSE(cdatasub["flows"],cdatasub["doubsimfitted"])

117.622

In [ ]:
# Run a doubly constrained SIM with a negative exponential cost function.
#dbl_form = 'flows ~ flows + station_origin + log_distance-1'
doubsim_form = "flows ~ station_origin + station_destination + distance -1"
doubsim1 = smf.glm(formula=doubsim_form, data = cdatasub, family = sm.families.Poisson()).fit()
print(doubsim1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                43945
Model:                            GLM   Df Residuals:                    43149
Model Family:                 Poisson   Df Model:                          795
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -7.6208e+05
Date:                Mon, 09 May 2022   Deviance:                   1.3520e+06
Time:                        10:00:56   Pearson chi2:                 1.74e+06
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [ ]:
cdatasub["doubsimfitted1"] = np.round(doubsim1.mu,0)

In [ ]:
CalcRSqaured(cdatasub["flows"],cdatasub["doubsimfitted1"])

0.49977786985381284

In [ ]:
CalcRMSE(cdatasub["flows"],cdatasub["doubsimfitted1"])

109.598